# *FLC* expression analysis

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

### Import data

In [3]:
PP2A = pd.read_table('20190911_PP2A.txt')
PP2A.head()

,Pos,Name,Rep,Cp
0,A23,Ait-14,#1,27.92
1,B23,Ait-14,#1,27.86
2,C23,Ait-14,#1,27.91
3,J7,Ait-14,#2,28.17
4,K7,Ait-14,#2,28.96


In [4]:
FLC = pd.read_table('20190911_FLC.txt')
FLC.head()

,Pos,Name,Rep,Cp
0,A24,Ait-14,#1,24.08
1,B24,Ait-14,#1,24.16
2,C24,Ait-14,#1,24.01
3,J8,Ait-14,#2,24.14
4,K8,Ait-14,#2,24.23


### Get the median for each technical replicate

In [5]:
PP2A_median = PP2A.groupby(['Name','Rep']).agg({'Cp':'median'}).reset_index()
PP2A_median.head()

,Name,Rep,Cp
0,Ait-14,#1,27.91
1,Ait-14,#2,28.47
2,Ait-14,#3,26.23
3,Ait-9,#1,27.38
4,Ait-9,#2,29.14


In [6]:
FLC_median = FLC.groupby(['Name','Rep']).agg({'Cp':'median'}).reset_index()
FLC_median.head()

,Name,Rep,Cp
0,Ait-14,#1,24.08
1,Ait-14,#2,24.14
2,Ait-14,#3,22.06
3,Ait-9,#1,24.34
4,Ait-9,#2,25.88


### Merge the data

In [7]:
merged = pd.merge(FLC_median, PP2A_median,on=['Name','Rep'])
merged.head()

,Name,Rep,Cp_x,Cp_y
0,Ait-14,#1,24.08,27.91
1,Ait-14,#2,24.14,28.47
2,Ait-14,#3,22.06,26.23
3,Ait-9,#1,24.34,27.38
4,Ait-9,#2,25.88,29.14


In [8]:
merged.columns=['Accession','Replicate','Cp_FLC','Cp_PP2A']
merged.head()

,Accession,Replicate,Cp_FLC,Cp_PP2A
0,Ait-14,#1,24.08,27.91
1,Ait-14,#2,24.14,28.47
2,Ait-14,#3,22.06,26.23
3,Ait-9,#1,24.34,27.38
4,Ait-9,#2,25.88,29.14


### Set FRI+FLC+ as control for *FLC* and *PP2A* expression

In [9]:
FLC_control = merged.groupby('Accession').get_group('FRI+FLC+').Cp_FLC.median()
FLC_control

25.07

In [10]:
PP2A_control = merged.groupby('Accession').get_group('FRI+FLC+').Cp_PP2A.median()
PP2A_control

27.09

### Add difference between the control and the accession

In [11]:
merged['∆_FLC']=FLC_control - merged['Cp_FLC']
merged['∆_PP2A']=PP2A_control - merged['Cp_PP2A']
merged.head()

,Accession,Replicate,Cp_FLC,Cp_PP2A,∆_FLC,∆_PP2A
0,Ait-14,#1,24.08,27.91,0.99,-0.82
1,Ait-14,#2,24.14,28.47,0.93,-1.38
2,Ait-14,#3,22.06,26.23,3.01,0.86
3,Ait-9,#1,24.34,27.38,0.73,-0.29
4,Ait-9,#2,25.88,29.14,-0.81,-2.05


### Calculate ratio between FLC and PP2A per sample (with Pfaffl correction)
Primer efficiency after standard curve:
* **FLC**: 1.93
* **PP2A**: 1.889

In [12]:
merged['Ratio']=1.93**merged['∆_FLC']/1.889**merged['∆_PP2A']
merged.head()

,Accession,Replicate,Cp_FLC,Cp_PP2A,∆_FLC,∆_PP2A,Ratio
0,Ait-14,#1,24.08,27.91,0.99,-0.82,3.230070
1,Ait-14,#2,24.14,28.47,0.93,-1.38,4.433721
2,Ait-14,#3,22.06,26.23,3.01,0.86,4.187627
3,Ait-9,#1,24.34,27.38,0.73,-0.29,1.943409
4,Ait-9,#2,25.88,29.14,-0.81,-2.05,2.162590


### Plot

In [13]:
colorMap = ['gray','gray','gray',
            '#0075DC','#0075DC','#0075DC','#0075DC',
            '#FFA405','#FFA405','#FFA405','#FFA405',
            '#2BCE48','#2BCE48','#2BCE48','#2BCE48','#2BCE48','#2BCE48']

In [14]:
sns.reset_defaults()
sns.set(rc={'figure.figsize':(15,10),"axes.labelsize":30, 'font.size':30, 'xtick.labelsize': 15.0, 'ytick.labelsize': 20.0,})
sns.set_style('whitegrid')
fig, ax = plt.subplots()
boxprops = {'edgecolor': 'k', 'linewidth': 2,}
lineprops = {'color': 'k', 'linewidth': 2}
boxplot_kwargs = dict({'boxprops': boxprops, 'medianprops': lineprops,
                       'whiskerprops': lineprops, 'capprops': lineprops,
                       'width': 0.75,'palette': colorMap})
swarmplot_kwargs = dict({'linewidth': 0.6, 'size': 9},
                        )
sns.boxplot(x=merged["Accession"], 
            y="Ratio", 
            data=merged, 
            order=["FRI+FLC+", "FRI+FLC-", "Col-0", "Cvi-0", "S1-1", "S15-3", "S5-10", "F10-1-3", "F13-8", "F3-2", "F9-2", "Ait-14", "Ait-9", "Arb-0", "Elh-46", "Set-0", "Set-6"],
            linewidth=1.5,
            fliersize=0,
            ax=ax,
            **boxplot_kwargs)
for patch in ax.artists:
    r, g, b, a = patch.get_facecolor()
    patch.set_facecolor((r, g, b, .4))
sns.swarmplot(x=merged["Accession"], 
            y="Ratio", 
            data=merged, 
            order=["FRI+FLC+", "FRI+FLC-", "Col-0", "Cvi-0", "S1-1", "S15-3", "S5-10", "F10-1-3", "F13-8", "F3-2", "F9-2", "Ait-14", "Ait-9", "Arb-0", "Elh-46", "Set-0", "Set-6"],
            palette=colorMap,
            **swarmplot_kwargs)
ax.tick_params(labelrotation=90)
plt.xlabel('')
plt.ylabel('$\itFLC$ expression relative to FRI+FLC+',fontsize=20)
plt.savefig("FLCexpressionFigure.pdf", format="pdf",bbox_inches="tight")
# plt.show()

### Export data for statistical analysis

In [15]:
merged.to_csv('FLCexpression4stats.txt', columns = ['Accession','Ratio'],sep = '\t',index=False)

In [16]:
merged.to_csv('FLCexpression4paper.txt',sep = '\t',index=False)